In [8]:
import nest_asyncio
nest_asyncio.apply()
from pathlib import Path
from datetime import datetime


import sys
sys.path.append("../")

from desci_sense.shared_functions.init import init_multi_chain_parser_config
from desci_sense.shared_functions.parsers.multi_chain_parser import MultiChainParser
from desci_sense.evaluation.utils import get_dataset
from desci_sense.shared_functions.dataloaders import (
    scrape_post,
    convert_text_to_ref_post,
)

In [20]:
config = init_multi_chain_parser_config(llm_type="openai/gpt-3.5-turbo-0125",
                                        post_process_type="combined")

In [21]:
multi_chain_parser = MultiChainParser(config)

2024-05-10 12:50:03.642 | INFO     | desci_sense.shared_functions.parsers.multi_chain_parser:__init__:66 - Initializing MultiChainParser. PostProcessType=combined
2024-05-10 12:50:03.698 | INFO     | desci_sense.shared_functions.parsers.multi_chain_parser:__init__:73 - Initializing post parsers...
2024-05-10 12:50:03.699 | INFO     | desci_sense.shared_functions.parsers.post_parser_chain:__init__:26 - Initializing parser chain 'refs_tagger' 
2024-05-10 12:50:03.849 | INFO     | desci_sense.shared_functions.parsers.post_parser_chain:__init__:26 - Initializing parser chain 'multi_refs_tagger' 
2024-05-10 12:50:03.898 | INFO     | desci_sense.shared_functions.parsers.post_parser_chain:__init__:26 - Initializing parser chain 'topics' 
2024-05-10 12:50:03.949 | INFO     | desci_sense.shared_functions.parsers.post_parser_chain:__init__:26 - Initializing parser chain 'keywords' 
2024-05-10 12:50:03.991 | INFO     | desci_sense.shared_functions.parsers.post_parser_chain:__init__:26 - Initializ

## Loading wandb dataset to test

In [4]:
import wandb
import pandas as pd

wandb.login()

api = wandb.Api()

#TODO move from testing
run = wandb.init(project="testing", job_type="evaluation")

# get artifact path

dataset_artifact_id = (
        'common-sense-makers/filter_evaluation/labeled_tweets_no_threads:v1'
    )

# set artifact as input artifact
dataset_artifact = run.use_artifact(dataset_artifact_id)

# initialize table path
# add the option to call table_path =  arguments.get('--dataset')

# download path to table
a_path = dataset_artifact.download()
print("The path is",a_path)

# get dataset file name

table_path = Path(f"{a_path}/labeled_data_table_no_threads.table.json")


# return the pd df from the table
#remember to remove the head TODO
df = get_dataset(table_path)

table_path = Path(f"{a_path}/handles_chart.table.json")

df_handles = get_dataset(table_path)



Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shahar-r-oriel (common-sense-makers). Use `wandb login --relogin` to force relogin


wandb:   2 of 2 files downloaded.  


The path is /Users/shaharorielkagan/sensemakers/nlp/notebooks/artifacts/labeled_tweets_no_threads:v1


In [5]:
import concurrent.futures
from tqdm import tqdm

def process_text(text):
    return convert_text_to_ref_post(text)

# Assuming df['Text'] is your DataFrame column
with concurrent.futures.ThreadPoolExecutor() as executor:
    inputs = list(tqdm(executor.map(process_text, df['Text']), total=len(df['Text'])))


100%|██████████| 467/467 [00:40<00:00, 11.55it/s]


In [22]:
# batch process
results = multi_chain_parser.batch_process_ref_posts(inputs,active_list=["keywords", "topics"],batch_size=10)

2024-05-10 12:50:26.066 | WARNING  | desci_sense.shared_functions.web_extractors.citoid:before_retry:84 - Retry attempt 1
2024-05-10 12:50:26.067 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation_async_retry:104 - skipping citoid for https://twitter.com/post_liberal/status/1773125864521163168
2024-05-10 12:50:26.069 | WARNING  | desci_sense.shared_functions.web_extractors.citoid:before_retry:84 - Retry attempt 1
2024-05-10 12:50:26.069 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation_async_retry:111 - target_url=https://en.wikipedia.org/wiki/Einst%C3%BCrzende_Neubauten
2024-05-10 12:50:26.077 | WARNING  | desci_sense.shared_functions.web_extractors.citoid:before_retry:84 - Retry attempt 1
2024-05-10 12:50:26.078 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation_async_retry:111 - target_url=https://wiki.p2pfoundation.net/Category:Peerproduction#With_the_advent_of_the_P2P_Mode_of_Production%2C_the_com

  0%|          | 0/467 [00:00<?, ?it/s]

2024-05-10 12:51:36.077 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:batch_process_ref_posts:245 - Invoking parallel chain...


JSONDecodeError: Expecting value: line 7 column 1 (char 33)

In [17]:
# update predictions to dataframe
df['Predicted Label'] = [x.filter_classification.value for x in results]
df['Reasoning Steps'] = ["Keywords: "+x.debug['topics']['full_text']+"Topics: "+x.debug['keywords']['raw_text'] for x in results]
df['Keywords'] = [x.keywords for x in results]
df['Topics'] = [x.topics for x in results]
df['Ref item types'] = [x.item_types for x in results]



In [19]:
# Log dataframe in wandb
wandb.login()

api = wandb.Api()

#TODO move from testing
run = wandb.init(project="filter_evaluation", job_type="predictions")

# Create the evaluation artifact
current_datetime = datetime.now()
time = current_datetime.strftime("%Y%m%d%H%M%S")


artifact = wandb.Artifact("prediction_evaluation-" + str(time), type="evaluation")

# Create a wandb.Table from the Pandas DataFrame
table = wandb.Table(dataframe=df)

# Add the wandb.Table to the artifact
artifact.add(table, "prediction_evaluation")

# model_info is your model metadata
run.config.update(config)

 # Log the artifact
wandb.log_artifact(artifact, aliases=["latest"])

wandb.run.finish()

Python(56918) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(56919) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(56920) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(56921) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [ ]:
bad_posts = []
for i in range(20,len(df["Text"])):
    print(i)
    p = df['Text'][i]
    r = convert_text_to_ref_post(p)
    try:
        result=multi_chain_parser.process_ref_post(r,active_list=["topics",'keywords'])
    except Exception as e:
        print("Post with text: ",p)
        print('and url: ',df['postURL'][i])
        print("has raised an exception: ",e) 
        bad_posts.append((df['postURL'][i],e))
          


    

In [ ]:
print(bad_posts)

[]


Python(28593) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(28594) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(28595) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(28596) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(28597) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(28598) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(28599) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(28600) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(28601) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(28602) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(28603) Malloc